In [ ]:
# Core scientific stack
!pip install astroquery lightkurve astropy matplotlib numpy pandas tqdm


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astroquery.gaia import Gaia
import lightkurve as lk
from astropy.timeseries import LombScargle

print("All imports successful!")


In [ ]:
Gaia.ROW_LIMIT = 5

query = """
SELECT TOP 5
    source_id,
    ra,
    dec,
    phot_g_mean_mag,
    bp_rp
FROM gaiadr3.gaia_source
WHERE phot_g_mean_mag < 12
"""

job = Gaia.launch_job(query)  # SYNC query
gaia_results = job.get_results().to_pandas()

gaia_results


In [ ]:
# Select ONE Gaia source (first row)
row = gaia_results.iloc[0]

ra = row["ra"]
dec = row["dec"]

print(f"Using Gaia source at RA={ra:.4f}, DEC={dec:.4f}")


In [ ]:
# Search for TESS light curves near this position
search_result = lk.search_lightcurve(
    f"{ra} {dec}",
    mission="TESS",
    radius=0.001  # ~3.6 arcsec
)

search_result


In [ ]:
# Download the first available light curve
if len(search_result) == 0:
    print("No TESS data found for this Gaia source.")
else:
    lc = search_result[0].download()
    print(lc)


In [ ]:
if 'lc' in locals():
    lc = lc.remove_nans().normalize()
    lc.plot()
    plt.title("TESS Light Curve")
    plt.show()


In [ ]:
# Clean & normalize
lc_clean = lc.remove_nans().flatten(window_length=401).normalize()

lc_clean.plot()
plt.title("Flattened & Normalized TESS Light Curve")
plt.show()


In [ ]:
rms = np.std(lc_clean.flux.value)
print(f"RMS variability = {rms:.5f}")


In [ ]:
# Lomb-Scargle periodogram
frequency, power = LombScargle(
    lc_clean.time.value,
    lc_clean.flux.value
).autopower()

period = 1 / frequency

plt.figure(figsize=(7,4))
plt.plot(period, power)
plt.xlim(0, 10)  # days
plt.xlabel("Period (days)")
plt.ylabel("Power")
plt.title("Lomb–Scargle Periodogram")
plt.show()


In [ ]:
best_period = period[np.argmax(power)]
print(f"Strongest detected period ≈ {best_period:.3f} days")


In [ ]:
summary = pd.DataFrame({
    "gaia_source_id": [row["source_id"]],
    "ra": [ra],
    "dec": [dec],
    "tess_rms": [rms],
    "dominant_period_days": [best_period],
    "likely_instrumental": [best_period < 0.05]
})

summary


In [ ]:
summary.to_csv("gaia_tess_variability_summary.csv", index=False)


In [ ]:
len(gaia_results)


In [ ]:
def analyze_gaia_star(row, radius=0.001):
    ra, dec = row["ra"], row["dec"]

    try:
        search = lk.search_lightcurve(
            f"{ra} {dec}",
            mission="TESS",
            radius=radius
        )

        if len(search) == 0:
            return None

        lc = search[0].download()
        lc = lc.remove_nans().flatten(window_length=401).normalize()

        rms = np.std(lc.flux.value)

        frequency, power = LombScargle(
            lc.time.value,
            lc.flux.value
        ).autopower()

        period = 1 / frequency
        best_period = period[np.argmax(power)]

        return {
            "gaia_source_id": row["source_id"],
            "ra": ra,
            "dec": dec,
            "tess_rms": rms,
            "dominant_period_days": best_period,
            "likely_instrumental": best_period < 0.05
        }

    except Exception as e:
        print(f"Error for source {row['source_id']}: {e}")
        return None


In [ ]:
results = []

for _, row in gaia_results.iterrows():
    out = analyze_gaia_star(row)
    if out is not None:
        results.append(out)

results_df = pd.DataFrame(results)
results_df


In [ ]:
results_df.to_csv("gaia_tess_variability_catalog.csv", index=False)


In [ ]:
merged = gaia_results.merge(
    results_df,
    left_on="source_id",
    right_on="gaia_source_id",
    how="inner"
)

merged.head()


In [ ]:
print(merged.columns)
print(len(merged))


In [ ]:
# Clean column names
merged = merged.rename(columns={
    "ra_x": "ra",
    "dec_x": "dec"
})

# Drop redundant columns
merged = merged.drop(columns=["ra_y", "dec_y"])

merged


In [ ]:
plt.figure(figsize=(6,5))

sc = plt.scatter(
    merged["bp_rp"],
    merged["phot_g_mean_mag"],
    c=merged["tess_rms"],
    cmap="viridis",
    s=80,
    edgecolor="k"
)

plt.gca().invert_yaxis()
plt.colorbar(sc, label="TESS RMS variability")
plt.xlabel("BP − RP")
plt.ylabel("G magnitude")
plt.title("Gaia HR Diagram Colored by TESS Variability")

plt.show()


In [ ]:
def variability_class(rms):
    if rms < 0.001:
        return "Quiet"
    elif rms < 0.005:
        return "Mildly Variable"
    else:
        return "Strongly Variable"

merged["variability_class"] = merged["tess_rms"].apply(variability_class)

merged[[
    "source_id",
    "tess_rms",
    "dominant_period_days",
    "variability_class",
    "likely_instrumental"
]]


In [ ]:
merged["variability_class"].value_counts()


In [ ]:
# Gaia variability proxy (lower flux_over_error = more variable)
merged["gaia_variable_proxy"] = merged["phot_g_mean_mag"] > 0  # placeholder logic already filtered

pd.crosstab(
    merged["likely_instrumental"],
    merged["variability_class"],
    rownames=["Instrumental (TESS)"],
    colnames=["TESS Variability Class"]
)


In [ ]:
merged.to_csv("gaia_tess_variability_final_catalog.csv", index=False)


In [ ]:
plt.figure(figsize=(6,5))
sc = plt.scatter(
    merged["bp_rp"],
    merged["phot_g_mean_mag"],
    c=merged["tess_rms"],
    cmap="viridis",
    s=80
)
plt.gca().invert_yaxis()
plt.colorbar(sc, label="TESS RMS variability")
plt.xlabel("BP − RP")
plt.ylabel("G magnitude")
plt.title("Gaia HR Diagram Colored by TESS Variability")
plt.savefig("hr_diagram_tess_rms.png", dpi=300, bbox_inches="tight")
plt.close()
